# Predicting the theraphy / drug for a rare disease and the duration for taking the theraphy / drug.

In [ ]:
####### Reading the Data and Importing the required Python packages ##########

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score

### Reading Data ######

PatientData = pd.ExcelFile('E:/ZoomRx/ZoomRx - Patient Charts Data Science Case Study.xlsx')
PatientData_Training = pd.read_excel(PatientData, 'Training Data')
PatientData_Test = pd.read_excel(PatientData, 'Testing Data')

#### Reshaping the Training data #######
PatientData_Training =  PatientData_Training.drop(PatientData_Training.index[[0]])

PatientData_Training = PatientData_Training.reset_index(drop=True)

PatientData_Training.columns = PatientData_Training.iloc[0]

PatientData_Training =  PatientData_Training.drop(PatientData_Training.index[[0]])

PatientData_Training = PatientData_Training.reset_index(drop=True)

del PatientData_Training["Question ID -->"]

PatientData_Training.columns = ['Patient ID #','Date of survey completion','Q1_10_1', 'Q1_10_2',
'Q1_10_3', 'Q1_23', 'Q1_24', 'Q1_25_1', 'Q1_25_2', 'Q1_25_3', 'Q1_25_4',
'Q1_25_5', 'Q1_25_6', 'Q1_25_7', 'Q1_25_8', 'Q1_25_9', 'Q1_25_10',
'Q1_25_11', 'Q1_25_12', 'Q1_40', 'Q2_05', 'Q2_10', 'Q2_15', 'Q2_16',
'Q2_20', 'Q2_24', 'Q2_25', 'Q2_55_1', 'Q2_55_2', 'Q2_55_3', 'Q2_55_4',
'Q2_55_5', 'Q2_55_6', 'Q2_55_7', 'Q2_55_8', 'Q2_55_9', 'Q2_55_10',
'Q2_55_11', 'Q2_55_12', 'Q2_55_13', 'Q2_55_14', 'Q2_55_15', 'Q2_55_16',
'Q2_55_17', 'Q2_55_18', 'Q2_55_19', 'Q2_55_20', 'Q2_100', 'Q2_125_1',
'Q2_125_2', 'Q2_125_3', 'Q2_125_4', 'Q2_125_5', 'Q2_125_6', 'Q2_125_7',
'Q2_125_8', 'Q2_125_9', 'Q2_125_10', 'Q2_125_11', 'Q2_125_12',
'Q2_125_13', 'Q2_127', 'Q2_130', 'Q3_10', 'Q3_20', 'Q3_30', 'Q3_40_1',
'Q3_40_2', 'Q3_40_3', 'Q3_40_4','Q2_40','Q2_65']

#### Reshaping the Testing data #######
PatientData_Test =  PatientData_Test.drop(PatientData_Training.index[[0]])

PatientData_Test = PatientData_Test.reset_index(drop=True)

PatientData_Test.columns = PatientData_Test.iloc[0]

PatientData_Test =  PatientData_Test.drop(PatientData_Training.index[[0]])

PatientData_Test = PatientData_Test.reset_index(drop=True)

del PatientData_Test["Question ID -->"]

PatientData_Test.columns = ['Patient ID #','Date of survey completion','Q1_10_1', 'Q1_10_2',
'Q1_10_3', 'Q1_23', 'Q1_24', 'Q1_25_1', 'Q1_25_2', 'Q1_25_3', 'Q1_25_4',
'Q1_25_5', 'Q1_25_6', 'Q1_25_7', 'Q1_25_8', 'Q1_25_9', 'Q1_25_10',
'Q1_25_11', 'Q1_25_12', 'Q1_40', 'Q2_05', 'Q2_10', 'Q2_15', 'Q2_16',
'Q2_20', 'Q2_24', 'Q2_25', 'Q2_55_1', 'Q2_55_2', 'Q2_55_3', 'Q2_55_4',
'Q2_55_5', 'Q2_55_6', 'Q2_55_7', 'Q2_55_8', 'Q2_55_9', 'Q2_55_10',
'Q2_55_11', 'Q2_55_12', 'Q2_55_13', 'Q2_55_14', 'Q2_55_15', 'Q2_55_16',
'Q2_55_17', 'Q2_55_18', 'Q2_55_19', 'Q2_55_20', 'Q2_100', 'Q2_125_1',
'Q2_125_2', 'Q2_125_3', 'Q2_125_4', 'Q2_125_5', 'Q2_125_6', 'Q2_125_7',
'Q2_125_8', 'Q2_125_9', 'Q2_125_10', 'Q2_125_11', 'Q2_125_12',
'Q2_125_13', 'Q2_127', 'Q2_130', 'Q3_10', 'Q3_20', 'Q3_30', 'Q3_40_1',
'Q3_40_2', 'Q3_40_3', 'Q3_40_4','Q2_40','Q2_65']


# Drug / Theraphy Prediction

In [ ]:
##### Playing around the Training Data fro further analysis ######

In [ ]:
# Training Data without Q2_65 - The intended duration of the prescribed therapy
PatientData_Training_new = PatientData_Training.iloc[:,0:71]

PatientData_Training_new.columns
PatientData_Training_new.dtypes

# Deleting 'Date of survey completion','Q2_05'-The date when the patient was originally diagnosed with the rare
# disease,'Patient ID #' columns from the dataframe as they are not required for the prediction

del PatientData_Training_new['Date of survey completion']
del PatientData_Training_new['Q2_05']
del PatientData_Training_new['Patient ID #']

In [ ]:
#### Identifying the Dependent and Independent variables ######

In [ ]:
# Dependent Variable (DV) = 'Q2_40'
# Independent Variable (IDV) ='Q1_10_1', 'Q1_10_2',
#       'Q1_10_3', 'Q1_23', 'Q1_24', 'Q1_25_1', 'Q1_25_2', 'Q1_25_3', 'Q1_25_4',
#       'Q1_25_5', 'Q1_25_6', 'Q1_25_7', 'Q1_25_8', 'Q1_25_9', 'Q1_25_10',
#       'Q1_25_11', 'Q1_25_12', 'Q1_40', 'Q2_05', 'Q2_10', 'Q2_15', 'Q2_16',
#       'Q2_20', 'Q2_24', 'Q2_25', 'Q2_55_1', 'Q2_55_2', 'Q2_55_3', 'Q2_55_4',
#       'Q2_55_5', 'Q2_55_6', 'Q2_55_7', 'Q2_55_8', 'Q2_55_9', 'Q2_55_10',
#       'Q2_55_11', 'Q2_55_12', 'Q2_55_13', 'Q2_55_14', 'Q2_55_15', 'Q2_55_16',
#       'Q2_55_17', 'Q2_55_18', 'Q2_55_19', 'Q2_55_20', 'Q2_100', 'Q2_125_1',
#       'Q2_125_2', 'Q2_125_3', 'Q2_125_4', 'Q2_125_5', 'Q2_125_6', 'Q2_125_7',
#       'Q2_125_8', 'Q2_125_9', 'Q2_125_10', 'Q2_125_11', 'Q2_125_12',
#       'Q2_125_13', 'Q2_127', 'Q2_130', 'Q3_10', 'Q3_20', 'Q3_30', 'Q3_40_1',
#       'Q3_40_2', 'Q3_40_3', 'Q3_40_4'

In [ ]:
# Checking the correlation between the variables
PatientData_Training_CorrelationMatrix = PatientData_Training_new.corr()

In [ ]:
# Checking the diiferent class of theraphy  
Number_of_unique_Theraphy = list (set(PatientData_Training_new["Q2_40"]))

In [ ]:
# There are many categorical columns that has to be encoded

# Using LabelEncoder to encode categorical variables

# Getting the categorical columns
cols = PatientData_Training_new.columns
num_cols = PatientData_Training_new._get_numeric_data().columns
list(set(cols) - set(num_cols))

from sklearn.preprocessing import LabelEncoder
var_mod = [
 'Q2_25',
 'Q1_10_2',
 'Q3_20',
 'Q2_16',
 'Q2_20',
 'Q1_23',
 'Q2_24',
 'Q3_30',
 'Q1_40',
 'Q2_15',
 'Q2_10',
 'Q3_10',
 'Q1_10_3',
 'Q2_127',
 'Q2_130']
le = LabelEncoder()
for i in var_mod:
    mask = ~PatientData_Training_new[i].isnull()
    PatientData_Training_new[i][mask] = le.fit_transform(PatientData_Training_new[i][mask])

In [ ]:
# Filling "nan" with mean of each column
PatientData_Training_new = PatientData_Training_new.fillna(PatientData_Training_new.mean())

In [ ]:
# Exploratory Analysis using some variables to see the distinction between the drug/theraphy classes

sns.lmplot("Q1_10_1","Q2_55_3", data = PatientData_Training_new, 
           hue = "Q2_40", fit_reg = False )

PatientData_Training_new.boxplot(column = "Q1_10_1", by = "Q2_40") 
PatientData_Training_new.boxplot(column = "Q2_20", by = "Q2_40") 

In [ ]:
# Training - Test Split of "PatientData_Training_new" data for building classification models for drug prediction on Test data

X_PDT_train, X_PDT_test, y_PDT_train, y_PDT_test = \
    train_test_split(PatientData_Training_new.iloc[:,0:67], 
                     PatientData_Training_new.iloc[:,67], 
                     test_size=0.3, random_state= 30)
y_PDT_train.value_counts()
y_PDT_test.value_counts()

# Building KNN Model for Drug Classification

In [ ]:
# Building K Nearest Neighbors model on training data
from sklearn.neighbors import KNeighborsClassifier
Prescribed_theraphy_knn = KNeighborsClassifier().fit(X_PDT_train,y_PDT_train)

# Theraphy Prediction based on the KNN model built on training data
predicted_Theraphy_knn = Prescribed_theraphy_knn.predict(X_PDT_test)
actual_Theraphy_knn = y_PDT_test

# Confusion matrix
pd.crosstab(actual_Theraphy_knn,predicted_Theraphy_knn,
            rownames = ["Actual Theraphy Class"],
            colnames = ["Predicted Theraphy Class"])

# Checking Accuracy Score
accuracy_score(actual_Theraphy_knn,predicted_Theraphy_knn)
# 69.03 %

In [ ]:
# Checking Accuracy for different K values

accuracy_all = pd.Series([0.0]*20, index = range(1,21))
for k in range(1,21):
    Prescribed_theraphy_knn = KNeighborsClassifier(n_neighbors = k).fit(X_PDT_train,y_PDT_train)
    predicted_Theraphy_knn = Prescribed_theraphy_knn.predict(X_PDT_test)
    accuracy_all[k] = accuracy_score(y_PDT_test,predicted_Theraphy_knn)

accuracy_all_rnd_30 = accuracy_all.copy()
# Accuracy is max of 69.03% at K= 5

# Trying to improve accuracy further by reducing dimentionality with Principal Component Analysis [PCA]

In [ ]:
from sklearn.decomposition import PCA

newPTD = PatientData_Training_new.iloc[:,0:67]
newPTD_corr = newPTD.corr()

# Scaling

newPTD_scaled= pd.DataFrame(scale(newPTD))
newPTD_scaled.columns = newPTD.columns

# Correlation comparison between scaled and unscaled data:
newPTD_scaled_corr =  newPTD_scaled.corr()
newPTD_corr = newPTD.corr()
# correlation doesn't get affected due to scaling

# PCA Model buildng
newPTDpca = PCA().fit(newPTD_scaled)
newPTD_projected = pd.DataFrame(newPTDpca.transform(newPTD_scaled))

# Dim1, Dim2, Dim3.... Dim67
newPTD_projected.columns = ["Dim" + str(i) for i in range(0,67)]

# Explained Variance Ratio
sum(newPTDpca.explained_variance_ratio_)
np.cumsum(newPTDpca.explained_variance_ratio_)
# With 35 variables we will get more than 70% variance in the data

# Building different Classification Models on the PCA Projected data and checking accuracy

# KNN, Gaussian Naive Baye's, Decision Tree, Random Forest 

In [ ]:
PTD_New = newPTD_projected.iloc[:,0:35]
PTD_New["Theraphy"] = PatientData_Training_new.iloc[:,67]

X_PDT_train, X_PDT_test, y_PDT_train, y_PDT_test = \
    train_test_split(PTD_New.iloc[:,0:35], 
                     PTD_New.iloc[:,35], 
                     test_size=0.3, random_state= 30)
y_PDT_train.value_counts()
y_PDT_test.value_counts()

################### KNN ###################################################################

Prescribed_theraphy_knn = KNeighborsClassifier(n_neighbors = 5,weights= 'distance', algorithm= 'brute').fit(X_PDT_train,y_PDT_train)

# Theraphy Prediction based on the KNN model built on training data
predicted_Theraphy_knn = Prescribed_theraphy_knn.predict(X_PDT_test)
actual_Theraphy_knn = y_PDT_test

# Confusion matrix
pd.crosstab(actual_Theraphy_knn,predicted_Theraphy_knn,
            rownames = ["Actual Theraphy Class"],
            colnames = ["Predicted Theraphy Class"])

accuracy_score(actual_Theraphy_knn,predicted_Theraphy_knn)
# 71.34%

np.mean(cross_val_score(KNeighborsClassifier(n_neighbors = 5,weights= 'distance', algorithm= 'brute'),
                PTD_New.iloc[:,0:35], PTD_New.iloc[:,35], 
                cv = 10))

# Cross Validation Score: 70.55%

################ Gaussian Naive Baye's#######################################################################

from sklearn.naive_bayes import GaussianNB
PD_nb = GaussianNB().fit(X_PDT_train,y_PDT_train)
predicted_Theraphy_nb = PD_nb.predict(X_PDT_test)
pd.crosstab(y_PDT_test,predicted_Theraphy_nb,
            rownames = ["Actual Theraphy Class"],
            colnames = ["Predicted Theraphy Class"])

accuracy_score(y_PDT_test,predicted_Theraphy_nb)

#51.16%

############# Decision Tree ################################################################################

from sklearn.tree import DecisionTreeClassifier
np.mean(cross_val_score(DecisionTreeClassifier(),
               PTD_New.iloc[:,0:35], PTD_New.iloc[:,35], 
                cv = 10))
#58.03%

############# Random Forest #################################################################################

from sklearn.ensemble import RandomForestClassifier
np.mean(cross_val_score(RandomForestClassifier(),
               PTD_New.iloc[:,0:35], PTD_New.iloc[:,35], 
                cv = 10))
#68.8%

In [ ]:
### PCA didn't help much with improving accuracy, rather reduced the accuracy as there are many categorical variables. 
### So we infer that PCA is insignificant on categorical variables

# Trying to improve accuracy by building Ensemble Models - Gradient Boosting directly on the Training Data without PCA

In [ ]:
# Improving Accuracy with Ensemble Methods - Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

PD_gb = GradientBoostingClassifier().fit(X_PDT_train,y_PDT_train)
predicted_Theraphy_nb = PD_gb.predict(X_PDT_test)
pd.crosstab(y_PDT_test,predicted_Theraphy_nb,
            rownames = ["Actual Theraphy Class"],
            colnames = ["Predicted Theraphy Class"])

accuracy_score(y_PDT_test,predicted_Theraphy_nb)
# 85.44%

In [ ]:
# Gradient Boosting has significantly increased the accuracy to 85.44%. 
# Therefore, we proceed with Gradient Boosting model for predicting the Drug / Theraphy on Test Data

# A. Predicting the Drugs to be prescribed per patient for the rare disease

In [ ]:
####################### Test Data Theraphy Predictions ##########################
##### (a) the drug that apatient suffering from this rare disease is most likely to receive ######


PatientData_Test_new = PatientData_Test.iloc[:,0:71]

PatientData_Test_new.columns

del PatientData_Test_new['Date of survey completion']
del PatientData_Test_new['Q2_05']
del PatientData_Test_new['Patient ID #']


# Using LabelEncoder to encode categorical variables

# Getting the categorical columns
cols = PatientData_Test_new.columns
num_cols = PatientData_Test_new._get_numeric_data().columns
list(set(cols) - set(num_cols))

from sklearn.preprocessing import LabelEncoder
var_mod = [
 'Q2_25',
 'Q1_10_2',
 'Q3_20',
 'Q2_16',
 'Q2_20',
 'Q1_23',
 'Q2_24',
 'Q3_30',
 'Q1_40',
 'Q2_15',
 'Q2_10',
 'Q3_10',
 'Q1_10_3',
 'Q2_127',
 'Q2_130']
le = LabelEncoder()
for i in var_mod:
    mask = ~PatientData_Test_new[i].isnull()
    PatientData_Test_new[i][mask] = le.fit_transform(PatientData_Test_new[i][mask])

# Filling "nan" with mean of each column
PatientData_Test_new = PatientData_Test_new.fillna(PatientData_Test_new.mean())

newPTD_test = PatientData_Test_new.iloc[:,0:67]

predicted_Test_Theraphy_gb = PD_gb.predict(newPTD_test)

### Final Predicted Prescribed Theraphy 
print(predicted_Test_Theraphy_gb)

PatientData_Test_new["Q2_40"] = predicted_Test_Theraphy_gb
PatientData_Test["Q2_40"] = predicted_Test_Theraphy_gb ### Theraphy Predicted with 85.44% accuracy

# Theraphy/ Drug Duration Prediction including Theraphy as IDV

In [ ]:
# Now we will have to consider the Drug / theraphy "Q2_40" as an independent variable for predicting the duration for which the drug has to be given

In [ ]:
Training_Theraphy_vs_Duration = pd.crosstab(PatientData_Training["Q2_40"],PatientData_Training["Q2_65"],
            rownames = ["Actual Theraphy Class"],
            colnames = ["Predicted Theraphy Class"])
PatientData_Training["Q2_65"].dtypes
PatientData_Training.dtypes

PD_train_including_theraphy = PatientData_Training.copy()

del PD_train_including_theraphy['Date of survey completion']
del PD_train_including_theraphy['Q2_05']
del PD_train_including_theraphy['Patient ID #']

# Dependent Variable (DV) = 'Q2_65'
# Independent Variable (IDV) = ALl other variables including Q2_40 - Theraphy Prescribed

from sklearn.preprocessing import LabelEncoder
var_mod = [
 'Q2_40',
 'Q2_25',
 'Q1_10_2',
 'Q3_20',
 'Q2_16',
 'Q2_20',
 'Q1_23',
 'Q2_24',
 'Q3_30',
 'Q1_40',
 'Q2_15',
 'Q2_10',
 'Q3_10',
 'Q1_10_3',
 'Q2_127',
 'Q2_130']
le = LabelEncoder()
for i in var_mod:
    mask = ~PD_train_including_theraphy[i].isnull()
    PD_train_including_theraphy[i][mask] = le.fit_transform(PD_train_including_theraphy[i][mask])

# Filling "nan" with mean of each column
PD_train_including_theraphy = PD_train_including_theraphy.fillna(PD_train_including_theraphy.mean()) 

In [ ]:
# Training - Test Split of "PD_train_including_theraphy" data

X_PDT_train, X_PDT_test, y_PDT_train, y_PDT_test = \
    train_test_split(PD_train_including_theraphy.iloc[:,0:68], 
                     PD_train_including_theraphy.iloc[:,68], 
                     test_size=0.3, random_state= 30)
y_PDT_train.value_counts()
y_PDT_test.value_counts()

# Building KNN on Training data to predict the duration of drug

In [ ]:
# Building K Nearest Neighbors model on training data

Prescribed_theraphy_duration_knn = KNeighborsClassifier().fit(X_PDT_train,y_PDT_train)

# Theraphy Prediction based on the KNN model built on training data
predicted_theraphy_duration_knn = Prescribed_theraphy_duration_knn.predict(X_PDT_test)
actual_Theraphy_duration_knn = y_PDT_test

# Confusion matrix
pd.crosstab(actual_Theraphy_duration_knn,predicted_theraphy_duration_knn,
            rownames = ["Actual Theraphy Duration Class"],
            colnames = ["Predicted Theraphy Duration Class"])

accuracy_score(actual_Theraphy_duration_knn,predicted_theraphy_duration_knn)
# 75.27 %

# Improving Accuracy with Gradient Boosting

In [ ]:
################# Improving the Accuracy with Ensemble Methods - Gradient Boosting ###############
from sklearn.ensemble import GradientBoostingClassifier
Theraphy_duration_gb = GradientBoostingClassifier().fit(X_PDT_train,y_PDT_train)
predicted_Theraphy_duration_nb = Theraphy_duration_gb.predict(X_PDT_test)
pd.crosstab(y_PDT_test,predicted_Theraphy_duration_nb,
            rownames = ["Actual Theraphy Class"],
            colnames = ["Predicted Theraphy Class"])

accuracy_score(y_PDT_test,predicted_Theraphy_duration_nb)
# 86.32%                   

In [ ]:
# Gradient Boosting has significantly increased the accuracy to 86.32%. 
# Therefore, we proceed with Gradient Boosting model for predicting the Drug Duration on Test Data

# B. Predicting the Duration for which the predicted Drug has to be afdministered

In [ ]:
##### Theraphy Duration Prediction on Test data ####

PatientData_test_TheraphyDuration = PatientData_Test.copy()

del PatientData_test_TheraphyDuration['Date of survey completion']
del PatientData_test_TheraphyDuration['Q2_05']
del PatientData_test_TheraphyDuration['Patient ID #']

# Dependent Variable (DV) = 'Q2_65'
# Independent Variable (IDV) = ALl other variables including Q2_40 - Theraphy Prescribed

from sklearn.preprocessing import LabelEncoder
var_mod = [
 'Q2_40',
 'Q2_25',
 'Q1_10_2',
 'Q3_20',
 'Q2_16',
 'Q2_20',
 'Q1_23',
 'Q2_24',
 'Q3_30',
 'Q1_40',
 'Q2_15',
 'Q2_10',
 'Q3_10',
 'Q1_10_3',
 'Q2_127',
 'Q2_130']
le = LabelEncoder()
for i in var_mod:
    mask = ~PatientData_test_TheraphyDuration[i].isnull()
    PatientData_test_TheraphyDuration[i][mask] = le.fit_transform(PatientData_test_TheraphyDuration[i][mask])

# Filling "nan" with mean of each column
PatientData_test_TheraphyDuration = PatientData_test_TheraphyDuration.fillna(PatientData_test_TheraphyDuration.mean()) 

PTD_test_TheraphyDuration = PatientData_test_TheraphyDuration.iloc[:,0:68]

predicted_Test_Theraphy_Duration_gb = Theraphy_duration_gb.predict(PTD_test_TheraphyDuration)

### Final Predicted Prescribed Theraphy 
print(predicted_Test_Theraphy_Duration_gb)

PatientData_test_TheraphyDuration["Q2_65"] = predicted_Test_Theraphy_Duration_gb
PatientData_Test["Q2_65"] = predicted_Test_Theraphy_Duration_gb

# Final Predicted Theraphy/Drug and Duration

In [ ]:
##### Dataframe of Final Predicted Theraphy and Duration ###############################

Final_Predicted_Theraphy_Duration_on_Testdata = PatientData_Test.copy()